In [4]:
from elasticsearch import Elasticsearch
from ssl import create_default_context
from datetime import datetime, timedelta
import ssl
import pandas as pd
import warnings
import pytz
import numpy as np


In [5]:
from elasticsearch import Elasticsearch, helpers
import os, uuid

In [6]:
# Set up an insecure SSL context
context = create_default_context()
context.check_hostname = False
context.verify_mode = ssl.CERT_NONE

In [7]:
# Instantiate the Elasticsearch class to connect to our server
es = Elasticsearch(
    ['elasticsearch-wazuh.apps.prod.racloud.globalinfotek.com'],   
    http_auth=('admin', 'SecretPassword'),
    scheme="https",
    port=443,
    ssl_context=context
)

In [8]:
# es.info()
# Set ES index(es) to use
es_index = "wazuh-alerts-*"

In [9]:
# timezone must be a valid pytz timezone string
timezone = 'US/Eastern'
tz = pytz.timezone(timezone)

In [10]:
# Set rule level range for ES query
min_rule_level = 0
max_rule_level = 21

In [11]:
# Minimum rule fired times to filter out infrequent events
min_firedtimes = 0

In [12]:
# Set age of data to pull as of the current time
MAX_EVENT_AGE_SECONDS = 0
MAX_EVENT_AGE_MINUTES = 0
MAX_EVENT_AGE_HOURS = 1
MAX_EVENT_AGE_DAYS = 0

In [13]:
# Convert the times to the correct date in the past
start_datetime = datetime.now() - \
    timedelta(
        days=MAX_EVENT_AGE_DAYS,
        hours=MAX_EVENT_AGE_HOURS,
        minutes=MAX_EVENT_AGE_MINUTES,
        seconds=MAX_EVENT_AGE_SECONDS,
)

In [14]:
# Set up the body of the es query.  Limiting results to only the first 10,000
body = {
    "from" : 0,
    "size" : 10000,
    "track_total_hits": True,
    "query": {
        "bool": { 
            "must": [
                {
                   "range": {
                        "rule.level": {
                            "gte": min_rule_level,
                            "lte": max_rule_level
                        }
                    }
                },
                {
                   "range": {
                     #   "rule.firedtimes": {
                     #       "gte": min_firedtimes,
                     #   }
                    }
                },
            ],
            
            "filter": [ 
                # { 
                #     "term":  {
                #         "agent.name": "HIPSEUT1-VeloCloudEdge"
                #     }
                # },
                {
                    "range": {
                        "timestamp": { 
                            "gte": start_datetime.strftime("%Y-%m-%dT%H:%M:%S")
                        }
                    },
                }
            ]
        }
    },
    "sort": [
        {
            # "rule.firedtimes": {
            #     "order": "desc"
            # },
            "timestamp": {
                "order": "desc"
            }
        },
    ],
}
      
    
    
    
    

In [15]:
# Establish a path filter for the es query to prevent downloading extra fields per record
filter_path = [
    'hits.total.value',
     # 'hits.hits._source.rule*',
    'hits.hits._source.rule.firedtimes',
    'hits.hits._source.rule.description',
    'hits.hits._source.timestamp',
    'hits.hits._source.agent.name',
    'hits.hits._source.rule.level',
    'hits.hits._source.rule.nist_800_53',
    'hits.hits._source.location'
]

In [16]:
# Run the query and store the results
results = es.search(
    index=es_index,
    body=body,
    filter_path=filter_path
)

ConnectionError: ConnectionError((<urllib3.connection.HTTPSConnection object at 0x00000206E1562A90>, 'Connection to elasticsearch-wazuh.apps.prod.racloud.globalinfotek.com timed out. (connect timeout=10)')) caused by: ConnectTimeoutError((<urllib3.connection.HTTPSConnection object at 0x00000206E1562A90>, 'Connection to elasticsearch-wazuh.apps.prod.racloud.globalinfotek.com timed out. (connect timeout=10)'))

In [ ]:
# Get the total number of results returned
total = int(results['hits']['total']['value'])

NameError: name 'results' is not defined

In [21]:
# import pandas lib as pd
import pandas as pd
df1 = pd.read_excel(r'C:\Users\dpandian\Documents\DataForDS.csv')
print(df1)

ValueError: Excel file format cannot be determined, you must specify an engine manually.